# Note book to create html pages for countries and Kreise in Germany

In [2]:
%config InlineBackend.figure_formats = ['svg']
%cp -v ../coronavirus.py .
from coronavirus import *
import datetime

../coronavirus.py -> ./coronavirus.py


In [3]:
d, c = fetch_deaths(), fetch_cases()

countries = d.index
countries2 = c.index
assert (countries2 == countries).all()

Downloaded data: last data point 4/8/20 from https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv
Downloaded data: last data point 4/8/20 from https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv


In [4]:
def modify_template(templatefile, output_file_name, mappings, wwwroot):
    """Create concrete *.ipynb file from template
    - templatefile: the template with placeholders to be substituted
    - mappings: dictiorany with placeholders as keys, and values to be substituted
    - output_file_name: name to write modified file to
    - wwwroot: directory in which the output file should be written
    """
    # open template
    with open(templatefile, "tr") as f_template:
        template = f_template.read()
    for key in mappings:
        template = template.replace(key, mappings[key])
    with open(os.path.join(wwwroot, output_file_name), "tw") as f:
        f.write(template)
    print(f"Written file to {output_file_name}")


In [ ]:
def check_country_name_is_known(name):
    d = fetch_deaths()
    assert name in d.index, f"{name} is unknown. Known countries are {sorted(d.index)}"

def germany_check_region_name_is_known(name):
    d = fetch_data_germany()
    assert name in list(d['Bundesland'].drop_duplicates()), \
        f"{name} is unknown. Known regions are {sorted(list(d['Bundesland'].drop_duplicates()))}"

def germany_check_subregion_name_is_known(name):
    d = fetch_data_germany()
    assert name in list(d['Landkreis'].drop_duplicates()), \
        f"{name} is unknown. Known regions are {sorted(list(d['Landkreis'].drop_duplicates()))}"

germany_check_region_name_is_known("Hamburg") 
germany_check_subregion_name_is_known("SK Hamburg") 

    
def sanitise(name):
    """Given a country name as a string, sanitise it for use as URL and filename: 
    - get rid of spaces, commas
    
    return cleaned string.
    
    (Leave umlaute for now)
    """
    s = name.replace(" ", "-")
    s = s.replace(",", "-")
    return s
    
    
def get_binder_url(notebook):
    base = "https://mybinder.org/v2/gh/fangohr/coronavirus/master?filepath="
    return base + notebook.replace(" ", "%20")


def create_ipynb_for_country(country, templatename, wwwroot):
    check_country_name_is_known(country)
    
    output_file_name =  f"{country}.ipynb"
    output_file_path = os.path.join(wwwroot, output_file_name)
    
    # country = sanitize(country)
    mappings = {
        "%title%" : country,
        "%title2%" : "",
        "%country%" : country,
        "%binderurl%" : get_binder_url(output_file_name),
        "%create_date%" : datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    }

    modify_template(templatename, output_file_name, mappings, wwwroot)
    assert os.path.exists(output_file_path)
    return output_file_name


Please be patient - downloading data from https://opendata.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0.csv ...


In [5]:
import subprocess
def nbconvert_ipynb2html(ipynb_path, wwwroot):
    
    # copy file to run the notebook
    command = f"cp -fv ../coronavirus.py {wwwroot}"
    subprocess.check_call(command, shell=True)
    
    # copy requirements (needed for binder)
    command = f"cp -fv ../requirements.txt {wwwroot}"
    subprocess.check_call(command, shell=True)
    
    # execute notebook and create html copy from it
    command = f"""jupyter-nbconvert "{os.path.join(wwwroot, ipynb_path)}" """ + \
                " --to html --execute"
    print(f"Command = {command}")
    output = subprocess.check_call(command, shell=True)

    # compute output path
    output_file_name = os.path.splitext(ipynb_path)[0] + ".html"
    assert os.path.exists(os.path.join(wwwroot, output_file_name))
    
    return output_file_name
    

In [6]:
def create_md_index_list(title, links):
    lines = []
    lines.append(title)
    lines.append("")   # need empty line for markdown syntax
    for name, (path_html, path_ipynb) in links.items():
        lines.append(f"* [{name}]({path_html})")
    return "\n".join(lines)
        

In [7]:
def get_country_list():
    d, c = fetch_deaths(), fetch_cases()

    countries = d.index
    countries2 = c.index
    assert (countries2 == countries).all()
    
    # Here we should identify regions in countries, and process those.
    # Instead, as a quick hack to get started, we'll just take one country
    # and the current "get_country" method will sum over all regions of one country if only 
    # the country name is given.
    
    return sorted(countries.drop_duplicates())
    

In [8]:
def create_html_for_john_hopkins_countries(countries, wwwroot):
    start_time = time.time()
    if not os.path.exists(wwwroot):
        print("To put the html into github repo for webhosting, run")
        print('"git clone git@github.com:fangohr/coronavirus.git wwwroot" or similar')
        os.mkdir(wwwroot)
    

    created_files = {}
    country = "Italy"
 
    for i, country in enumerate(countries):
        print(f"Processing {i+1}/{len(countries)} [{time.time()-start_time:4.0f}s]")
        ipynb_path = create_ipynb_for_country(country, "template-country.ipynb", wwwroot=wwwroot)
        html_path = nbconvert_ipynb2html(os.path.join(ipynb_path), wwwroot=wwwroot)
        created_files[country] = html_path, ipynb_path
    print("Create {len(countries) notebooks and html versions in" + \
          "{time.time()-start_time} seconds}")
    return created_files

In [9]:
def create_index_page(sections, rootname, wwwroot):
    """Sections is dictionary: key is title, value is markdown text"""
    md_file = rootname + ".md"
    
    with open(os.path.join(wwwroot, md_file), "tw") as f:
        for section in sections:
            f.write(f"# {section}\n\n")
            f.write(sections[section])
    print(f"Written overview to {md_file}.")
    html_file = rootname + ".html"
    subprocess.check_call(f"pandoc -t html -o {os.path.join(wwwroot, html_file)} " +
                          f"{os.path.join(wwwroot, md_file)}", shell=True)
    return html_file

In [10]:
def get_germany_subregion_list():
    """returns list of subregions (Kreise), 
    ordered according to (i) Land, then (ii) Kreis
    """
    x = fetch_data_germany()
    land_kreis = x[['Bundesland', 'Landkreis']]
    ordered = land_kreis.sort_values(['Bundesland', 'Landkreis'])
    return list(ordered['Landkreis'].drop_duplicates())
 

@joblib_memory.cache
def germany_get_bundesland_from_kreis(kreis):
        x = fetch_data_germany()
        return x[x['Landkreis'] == kreis].iloc[0]['Bundesland']    

# Create country overview for the world

In [11]:
wwwroot = "wwwroot"
countries = get_country_list()
created_files = create_html_for_john_hopkins_countries(countries, wwwroot)
index_md = create_md_index_list("Countries (data from Johns Hopkins University)", 
                                created_files)
intro ="[Additional plots for Germany](index-germany.html])\n\n"
create_index_page(sections={"Germany: Landkreise" : intro, 
                            "World" : index_md}, 
                  rootname="index", 
                  wwwroot=wwwroot)


To put the html into github repo for webhosting, run
"git clone git@github.com:fangohr/coronavirus.git wwwroot" or similar
Processing 1/184 [   0s]
Written file to Afghanistan.ipynb
Command = jupyter-nbconvert "wwwroot/Afghanistan.ipynb"  --to html --execute
Processing 2/184 [   8s]
Written file to Albania.ipynb
Command = jupyter-nbconvert "wwwroot/Albania.ipynb"  --to html --execute
Processing 3/184 [  14s]
Written file to Algeria.ipynb
Command = jupyter-nbconvert "wwwroot/Algeria.ipynb"  --to html --execute
Processing 4/184 [  20s]
Written file to Andorra.ipynb
Command = jupyter-nbconvert "wwwroot/Andorra.ipynb"  --to html --execute
Processing 5/184 [  26s]
Written file to Angola.ipynb
Command = jupyter-nbconvert "wwwroot/Angola.ipynb"  --to html --execute
Processing 6/184 [  32s]
Written file to Antigua and Barbuda.ipynb
Command = jupyter-nbconvert "wwwroot/Antigua and Barbuda.ipynb"  --to html --execute
Processing 7/184 [  37s]
Written file to Argentina.ipynb
Command = jupyter-nbco

Processing 62/184 [ 361s]
Written file to France.ipynb
Command = jupyter-nbconvert "wwwroot/France.ipynb"  --to html --execute
Processing 63/184 [ 367s]
Written file to Gabon.ipynb
Command = jupyter-nbconvert "wwwroot/Gabon.ipynb"  --to html --execute
Processing 64/184 [ 372s]
Written file to Gambia.ipynb
Command = jupyter-nbconvert "wwwroot/Gambia.ipynb"  --to html --execute
Processing 65/184 [ 378s]
Written file to Georgia.ipynb
Command = jupyter-nbconvert "wwwroot/Georgia.ipynb"  --to html --execute
Processing 66/184 [ 384s]
Written file to Germany.ipynb
Command = jupyter-nbconvert "wwwroot/Germany.ipynb"  --to html --execute
Processing 67/184 [ 390s]
Written file to Ghana.ipynb
Command = jupyter-nbconvert "wwwroot/Ghana.ipynb"  --to html --execute
Processing 68/184 [ 396s]
Written file to Greece.ipynb
Command = jupyter-nbconvert "wwwroot/Greece.ipynb"  --to html --execute
Processing 69/184 [ 402s]
Written file to Grenada.ipynb
Command = jupyter-nbconvert "wwwroot/Grenada.ipynb"  --

Processing 126/184 [ 730s]
Written file to North Macedonia.ipynb
Command = jupyter-nbconvert "wwwroot/North Macedonia.ipynb"  --to html --execute
Processing 127/184 [ 736s]
Written file to Norway.ipynb
Command = jupyter-nbconvert "wwwroot/Norway.ipynb"  --to html --execute
Processing 128/184 [ 742s]
Written file to Oman.ipynb
Command = jupyter-nbconvert "wwwroot/Oman.ipynb"  --to html --execute
Processing 129/184 [ 748s]
Written file to Pakistan.ipynb
Command = jupyter-nbconvert "wwwroot/Pakistan.ipynb"  --to html --execute
Processing 130/184 [ 754s]
Written file to Panama.ipynb
Command = jupyter-nbconvert "wwwroot/Panama.ipynb"  --to html --execute
Processing 131/184 [ 760s]
Written file to Papua New Guinea.ipynb
Command = jupyter-nbconvert "wwwroot/Papua New Guinea.ipynb"  --to html --execute
Processing 132/184 [ 766s]
Written file to Paraguay.ipynb
Command = jupyter-nbconvert "wwwroot/Paraguay.ipynb"  --to html --execute
Processing 133/184 [ 772s]
Written file to Peru.ipynb
Command 

'index.html'

In [11]:
print(create_md_index_list("Countries (data from Johns Hopkins University)", created_files))

Countries (data from Johns Hopkins University)

* [Afghanistan](Afghanistan.html)
* [Albania](Albania.html)
* [Algeria](Algeria.html)
* [Andorra](Andorra.html)
* [Angola](Angola.html)
* [Antigua and Barbuda](Antigua and Barbuda.html)
* [Argentina](Argentina.html)
* [Armenia](Armenia.html)
* [Australia](Australia.html)
* [Austria](Austria.html)
* [Azerbaijan](Azerbaijan.html)
* [Bahamas](Bahamas.html)
* [Bahrain](Bahrain.html)
* [Bangladesh](Bangladesh.html)
* [Barbados](Barbados.html)
* [Belarus](Belarus.html)
* [Belgium](Belgium.html)
* [Belize](Belize.html)
* [Benin](Benin.html)
* [Bhutan](Bhutan.html)
* [Bolivia](Bolivia.html)
* [Bosnia and Herzegovina](Bosnia and Herzegovina.html)
* [Botswana](Botswana.html)
* [Brazil](Brazil.html)
* [Brunei](Brunei.html)
* [Bulgaria](Bulgaria.html)
* [Burkina Faso](Burkina Faso.html)
* [Burma](Burma.html)
* [Burundi](Burundi.html)
* [Cabo Verde](Cabo Verde.html)
* [Cambodia](Cambodia.html)
* [Cameroon](Cameroon.html)
* [Canada](Canada.html)
* [Cen

# Create list of Germany data sets

In [14]:
@joblib_memory.cache
def create_html_for_Germany(subregions, wwwroot):
    start_time = time.time()
    created_files = {}

    if not os.path.exists(wwwroot):
        print("To put the html into github repo for webhosting, run")
        print('"git clone git@github.com:fangohr/coronavirus.git wwwroot" or similar')
        os.mkdir(wwwroot)

    for i, kreis in enumerate(subregions):
        bundesland = germany_get_bundesland_from_kreis(kreis)
        print(f"Processing {i+1}/{len(subregions)} [{time.time()-start_time:4.0f}s]")
        ipynb_path = create_ipynb_for_germany(region=bundesland, subregion=kreis, 
                                              templatename="template-germany.ipynb", wwwroot=wwwroot)
        html_path = nbconvert_ipynb2html(os.path.join(ipynb_path), wwwroot=wwwroot)
        one_line_summary = f"Germany: {bundesland} : {kreis}"
        created_files[one_line_summary] = html_path, ipynb_path
        print("Create {len(subregions) notebooks and html versions in" + \
              "{time.time()-start_time} seconds}")
    return created_files

In [15]:
def create_ipynb_for_germany(region, subregion, templatename, wwwroot):
    germany_check_region_name_is_known(region)
    germany_check_subregion_name_is_known(subregion)
    
    output_file_name =  f"Germany-{sanitise(region)}-{sanitise(subregion)}.ipynb"
    output_file_path = os.path.join(wwwroot, output_file_name)
    
    # country = sanitize(country)
    mappings = {
        "%title%" : f"Germany: {subregion} ({region})",
        "%title2%" : "",
        "%region%" : region,
        "%subregion%" : subregion,
        "%binderurl%" : get_binder_url(output_file_name),
        "%create_date%" : datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    }

    modify_template(templatename, output_file_name, mappings, wwwroot)
    assert os.path.exists(output_file_path)
    return output_file_name


In [17]:
wwwroot = "wwwroot"
subregions = get_germany_subregion_list()
created_files = create_html_for_Germany(subregions, wwwroot)
index_md = create_md_index_list("Landkreise in Germany", 
                                created_files)
create_index_page(sections={"Links to static pages for each Landkreis" : index_md}, 
                  rootname="index-germany", 
                  wwwroot=wwwroot)


Processing 1/412 [   0s]
Written file to Germany-Baden-Württemberg-LK-Alb-Donau-Kreis.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Baden-Württemberg-LK-Alb-Donau-Kreis.ipynb"  --to html --execute
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 2/412 [   8s]
Written file to Germany-Baden-Württemberg-LK-Biberach.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Baden-Württemberg-LK-Biberach.ipynb"  --to html --execute
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 3/412 [  16s]
Written file to Germany-Baden-Württemberg-LK-Bodenseekreis.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Baden-Württemberg-LK-Bodenseekreis.ipynb"  --to html --execute
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 4/412 [  24s]
Written file to Germany-Baden-Württemberg-LK-Breisgau-Hochschwarzwald.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Ba

Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 31/412 [ 235s]
Written file to Germany-Baden-Württemberg-LK-Sigmaringen.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Baden-Württemberg-LK-Sigmaringen.ipynb"  --to html --execute
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 32/412 [ 243s]
Written file to Germany-Baden-Württemberg-LK-Tuttlingen.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Baden-Württemberg-LK-Tuttlingen.ipynb"  --to html --execute
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 33/412 [ 251s]
Written file to Germany-Baden-Württemberg-LK-Tübingen.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Baden-Württemberg-LK-Tübingen.ipynb"  --to html --execute
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 34/412 [ 259s]
Written file to Germany-Baden-Württemberg-LK-

Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 62/412 [ 480s]
Written file to Germany-Bayern-LK-Donau-Ries.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Bayern-LK-Donau-Ries.ipynb"  --to html --execute
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 63/412 [ 487s]
Written file to Germany-Bayern-LK-Ebersberg.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Bayern-LK-Ebersberg.ipynb"  --to html --execute
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 64/412 [ 494s]
Written file to Germany-Bayern-LK-Eichstätt.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Bayern-LK-Eichstätt.ipynb"  --to html --execute
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 65/412 [ 502s]
Written file to Germany-Bayern-LK-Erding.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Bayern-LK-Erding.ipyn

Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 94/412 [ 712s]
Written file to Germany-Bayern-LK-Nürnberger-Land.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Bayern-LK-Nürnberger-Land.ipynb"  --to html --execute
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 95/412 [ 720s]
Written file to Germany-Bayern-LK-Oberallgäu.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Bayern-LK-Oberallgäu.ipynb"  --to html --execute
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 96/412 [ 727s]
Written file to Germany-Bayern-LK-Ostallgäu.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Bayern-LK-Ostallgäu.ipynb"  --to html --execute
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 97/412 [ 734s]
Written file to Germany-Bayern-LK-Passau.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Bayern-LK

Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 126/412 [ 946s]
Written file to Germany-Bayern-SK-Ingolstadt.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Bayern-SK-Ingolstadt.ipynb"  --to html --execute
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 127/412 [ 954s]
Written file to Germany-Bayern-SK-Kaufbeuren.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Bayern-SK-Kaufbeuren.ipynb"  --to html --execute
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 128/412 [ 961s]
Written file to Germany-Bayern-SK-Kempten.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Bayern-SK-Kempten.ipynb"  --to html --execute
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 129/412 [ 968s]
Written file to Germany-Bayern-SK-Landshut.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Bayern-SK-Landshu

...

Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 307/412 [2291s]
Written file to Germany-Rheinland-Pfalz-LK-Altenkirchen.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Rheinland-Pfalz-LK-Altenkirchen.ipynb"  --to html --execute
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 308/412 [2298s]
Written file to Germany-Rheinland-Pfalz-LK-Alzey-Worms.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Rheinland-Pfalz-LK-Alzey-Worms.ipynb"  --to html --execute
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 309/412 [2305s]
Written file to Germany-Rheinland-Pfalz-LK-Bad-Dürkheim.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Rheinland-Pfalz-LK-Bad-Dürkheim.ipynb"  --to html --execute
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 310/412 [2313s]
Written file to Germany-Rheinland-Pfalz-L

Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 336/412 [2502s]
Written file to Germany-Rheinland-Pfalz-SK-Neustadt-a.d.Weinstraße.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Rheinland-Pfalz-SK-Neustadt-a.d.Weinstraße.ipynb"  --to html --execute
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 337/412 [2509s]
Written file to Germany-Rheinland-Pfalz-SK-Pirmasens.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Rheinland-Pfalz-SK-Pirmasens.ipynb"  --to html --execute
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 338/412 [2516s]
Written file to Germany-Rheinland-Pfalz-SK-Speyer.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Rheinland-Pfalz-SK-Speyer.ipynb"  --to html --execute
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 339/412 [2523s]
Written file to Germany-Rheinland-P

Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 367/412 [2727s]
Written file to Germany-Sachsen-Anhalt-LK-Mansfeld-Südharz.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Sachsen-Anhalt-LK-Mansfeld-Südharz.ipynb"  --to html --execute
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 368/412 [2734s]
Written file to Germany-Sachsen-Anhalt-LK-Saalekreis.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Sachsen-Anhalt-LK-Saalekreis.ipynb"  --to html --execute
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 369/412 [2742s]
Written file to Germany-Sachsen-Anhalt-LK-Salzlandkreis.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Sachsen-Anhalt-LK-Salzlandkreis.ipynb"  --to html --execute
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 370/412 [2749s]
Written file to Germany-Sachsen-Anhalt-

Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 397/412 [2944s]
Written file to Germany-Thüringen-LK-Nordhausen.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Thüringen-LK-Nordhausen.ipynb"  --to html --execute
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 398/412 [2951s]
Written file to Germany-Thüringen-LK-Saale-Holzland-Kreis.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Thüringen-LK-Saale-Holzland-Kreis.ipynb"  --to html --execute
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 399/412 [2959s]
Written file to Germany-Thüringen-LK-Saale-Orla-Kreis.ipynb
Command = jupyter-nbconvert "wwwroot/Germany-Thüringen-LK-Saale-Orla-Kreis.ipynb"  --to html --execute
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 400/412 [2966s]
Written file to Germany-Thüringen-LK-Saalfeld-Rudolstad

'index-germany.html'